<a href="https://colab.research.google.com/github/hinafarooq21/F1-Lap-Predictor--Capstone/blob/main/Additional_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Collecting Circuit Details

In [ ]:
# Importing all the libraries
# Establish website connection using the requests library
import requests
import pandas as pd
import numpy as np
# the BeautifulSoup library for scraping from the bs4 package
from bs4 import BeautifulSoup
# RegEx for pattern matching
import re

In [ ]:
url= 'https://www.racefans.net/f1-information/going-to-a-race/'
response = requests.get(url)
print(response.status_code)
soup = BeautifulSoup(response.content, 'html.parser')
print(soup.prettify)

200
<bound method Tag.prettify of <!DOCTYPE html>

<html class="no-js" lang="en-GB">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width" name="viewport"/>
<link href="https://gmpg.org/xfn/11" rel="profile"/>
<link href="https://www.racefans.net/xmlrpc.php" rel="pingback"/>
<link href="/apple-touch-icon.png" rel="apple-touch-icon"/>
<link href="/apple-touch-icon-precomposed.png" rel="apple-touch-icon-precomposed"/>
<!--[if lt IE 9]>
	<script src="https://www.racefans.net/wp-content/themes/twentyfifteen/js/html5.js"></script>
	<![endif]-->
<script>(function(html){html.className = html.className.replace(/\bno-js\b/,'js')})(document.documentElement);</script>
<meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots">
<style>img:is([sizes="auto" i], [sizes^="auto," i]) { contain-intrinsic-size: 3000px 1500px }</style>
<!-- This site is optimized with the Yoast SEO Premium plugin v24.3 (Yoast SEO v24.3) - https://yoast.com/wordpr

In [ ]:
# Finding the links to access the data
entry_content = soup.find_all('div', class_="entry-content")
links = []
for link in entry_content:
    links_tags = link.find_all('a', href=True)
    for link_tag in links_tags:
        if links_tags:
            links.append(f"https://www.racefans.net/{link_tag['href']}")
print(links)


['https://www.racefans.net//f1-information/going-to-a-race/yas-island-abu-dhabi-circuit-information/', 'https://www.racefans.net//groups/going-to-the-abu-dhabi-grand-prix-at-yas-marina/', 'https://www.racefans.net//f1-information/going-to-a-race/albert-park-melbourne-circuit-information/', 'https://www.racefans.net//groups/going-to-the-australian-grand-prix-at-melbourne/forum/', 'https://www.racefans.net//f1-information/going-to-a-race/red-bull-ring-circuit-information/', 'https://www.racefans.net//groups/going-to-the-austrian-grand-prix-at-red-bull-ring/forum/', 'https://www.racefans.net//f1-information/going-to-a-race/baku-city-circuit-track-information/', 'https://www.racefans.net/https://www.racefans.net/groups/going-to-the-european-grand-prix-at-baku-city-circuit/forum/', 'https://www.racefans.net//f1-information/going-to-a-race/bahrain-international-circuit-track-information/', 'https://www.racefans.net//groups/going-to-the-bahrain-grand-prix-at-bahrain-international-circuit/foru

In [ ]:
# Check the number of links
len(links)

67

In [ ]:
# There are 2 types of links, one for fan forums and the other for track information
# We need to extract the track information

pattern = r'information/*'

circuit_links = [item for item in links if re.search(pattern, item)]
print(circuit_links)

['https://www.racefans.net//f1-information/going-to-a-race/yas-island-abu-dhabi-circuit-information/', 'https://www.racefans.net//f1-information/going-to-a-race/albert-park-melbourne-circuit-information/', 'https://www.racefans.net//f1-information/going-to-a-race/red-bull-ring-circuit-information/', 'https://www.racefans.net//f1-information/going-to-a-race/baku-city-circuit-track-information/', 'https://www.racefans.net//f1-information/going-to-a-race/bahrain-international-circuit-track-information/', 'https://www.racefans.net//f1-information/going-to-a-race/spa-francorchamps-circuit-information/', 'https://www.racefans.net//f1-information/going-to-a-race/autodromo-jose-carlos-pace-carlos-pace-interlagos-circuit-information/', 'https://www.racefans.net//f1-information/going-to-a-race/circuit-gilles-villeneuve-montreal-circuit-information/', 'https://www.racefans.net//f1-information/going-to-a-race/shanghai-international-circuit-china/', 'https://www.racefans.net//f1-information/going-t

In [ ]:
len(circuit_links)

35

In [ ]:
all_circuit_data = []

# Labels to extract - These are the id's that appear on the information page
labels_to_search = [
    "Lap length", "Race laps", "Race distance", "Lap record*",
    "Fastest lap", "Maximum speed", "Distance from grid to turn one", "Longest flat-out section"
]

for link in circuit_links:
    response = requests.get(link)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Get circuit name
    circuit_name = soup.find('h1', {'class': 'entry-title'}).text.strip()
    # Get the table records
    table = soup.find('table', {'class': 'thin'})
    # Create a dictionary to store the circuit's data
    circuit_data = {"Circuit Name": circuit_name}

    if table:
        # Iterate through each row in the table
        for row in table.find_all('tr'):
            # Get all the (td) cells in this row as they store the text data
            cells = row.find_all('td')
            if cells:
                # The first cell of the row contains the label
                label = cells[0].text.strip()
                # Check if the label matches one of the labels we're looking for
                if label in labels_to_search:
                    # The second cell (or the next one) contains the value
                    value = cells[1].text.strip()
                    circuit_data[label] = value

    # Append the circuit data to the main list
    all_circuit_data.append(circuit_data)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_circuit_data)

df

,Circuit Name,Lap length,Race laps,Race distance,Lap record*,Fastest lap,Maximum speed,Distance from grid to turn one,Longest flat-out section
0,"Yas Marina, Abu Dhabi – circuit information",5.554km (3.451 miles),55,305.355km (189.739 miles),"1’40.279 (Sebastian Vettel, 2009)","1’34.779 (Lewis Hamilton, 2019, qualifying three)",339kph (210.645 mph),305m,1233m
1,"Albert Park, Melbourne – circuit information",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Red Bull Ring – circuit information,4.326km (2.688 miles),71,307.02km (190.773 miles),"1’05.609 (Carlos Sainz Jnr, 2020)","1’02.939 (Valtteri Bottas, 2020, qualifying th...",327.4kph (203.437 mph),216.94m,868m
3,Baku City Circuit – track information,6.003km (3.73 miles),51,306.051km (190.171 miles),"1’43.009 (Charles Leclerc, 2019)","1’40.495 (Valtteri Bottas, 2019, qualifying th...",337kph (209.402 mph),137.8m,2010m
4,Bahrain International Circuit – track information,5.412km (3.363 miles),57,308.238km (191.53 miles),"1’31.447 (Pedro de la Rosa, 2005)","1’27.264 (Lewis Hamilton, 2020, qualifying three)",329.6kph (204.804 mph),336.49m,1205m
5,Spa-Francorchamps – circuit information,7.004km (4.352 miles),44,308.052km (191.415 miles),"1’46.286 (Valtteri Bottas, 2018)","1’41.252 (Lewis Hamilton, 2020, qualifying three)",333kph (206.917mph),271m,2015m
6,"Interlagos, Autodromo Jose Carlos Pace circuit...",4.309km (2.677 miles),71,305.909km (190.083 miles),"1’11.473 (Juan Pablo Montoya, 2004)","1’07.281 (Lewis Hamilton, 2018, qualifying three)",331kph (205.674 mph),195m,"1,394m"
7,"Circuit Gilles Villeneuve, Montreal – circuit ...",4.361km (2.71 miles),70,305.27km (189.686 miles),"1’13.622 (Rubens Barrichello, 2004)","1’10.764 (Sebastian Vettel, 2018, qualifying t...",315.5kph (196.043 mph),159.154m,1190m
8,Shanghai International Circuit information,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Silverstone – circuit information,5.891km (3.66 miles),52,306.332km (190.346 miles),"1’27.097 (Max Verstappen, 2020)","1’24.303 (Lewis Hamilton, 2020, qualifying three)",329.5kph (204.742 mph),296m,"1,034m"


In [ ]:
# Getting the list of columns
df.columns

Index(['Circuit Name', 'Lap length', 'Race laps', 'Race distance',
       'Lap record*', 'Fastest lap', 'Maximum speed',
       'Distance from grid to turn one', 'Longest flat-out section'],
      dtype='object')

In [ ]:
# Manually adding the missing information
# Shanghai
df.loc[8, ['Lap length', 'Race laps', 'Race distance',
       'Lap record*', 'Fastest lap', 'Maximum speed',
       'Distance from grid to turn one', 'Longest flat-out section']] = ['5.451km', '56', '305.006km', "1’32.238 (Michael Schumacher, 2004)","1’31.095 (Sebastian Vettel, 2018, qualifying three)", "348kph", "336m", '1397m']
# Imola
df.loc[11, ['Lap length', 'Race laps', 'Race distance',
       'Lap record*', 'Fastest lap', 'Maximum speed',
       'Distance from grid to turn one']] = ['4.909km', '63', '309.267km', "1’15.484 (Lewis Hamilton, 2020)","1’13.609 (Valtteri Bottas, 2020, qualifying three)", "325kph", "399m"]
# Valencia
df.loc[31, ['Maximum speed']] = ["318kph"]
# Istanbul Park
df.loc[32, ['Maximum speed']] = ["321.4kph"]
# Fuji Speedway
df.loc[34, ['Lap length', 'Race laps', 'Race distance', 'Maximum speed', 'Longest flat-out section']] = ["4.563km", "67", "305.416km","310kph","1445"]

In [ ]:
df

,Circuit Name,Lap length,Race laps,Race distance,Lap record*,Fastest lap,Maximum speed,Distance from grid to turn one,Longest flat-out section
0,"Yas Marina, Abu Dhabi – circuit information",5.554km (3.451 miles),55,305.355km (189.739 miles),"1’40.279 (Sebastian Vettel, 2009)","1’34.779 (Lewis Hamilton, 2019, qualifying three)",339kph (210.645 mph),305m,1233m
1,"Albert Park, Melbourne – circuit information",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Red Bull Ring – circuit information,4.326km (2.688 miles),71,307.02km (190.773 miles),"1’05.609 (Carlos Sainz Jnr, 2020)","1’02.939 (Valtteri Bottas, 2020, qualifying th...",327.4kph (203.437 mph),216.94m,868m
3,Baku City Circuit – track information,6.003km (3.73 miles),51,306.051km (190.171 miles),"1’43.009 (Charles Leclerc, 2019)","1’40.495 (Valtteri Bottas, 2019, qualifying th...",337kph (209.402 mph),137.8m,2010m
4,Bahrain International Circuit – track information,5.412km (3.363 miles),57,308.238km (191.53 miles),"1’31.447 (Pedro de la Rosa, 2005)","1’27.264 (Lewis Hamilton, 2020, qualifying three)",329.6kph (204.804 mph),336.49m,1205m
5,Spa-Francorchamps – circuit information,7.004km (4.352 miles),44,308.052km (191.415 miles),"1’46.286 (Valtteri Bottas, 2018)","1’41.252 (Lewis Hamilton, 2020, qualifying three)",333kph (206.917mph),271m,2015m
6,"Interlagos, Autodromo Jose Carlos Pace circuit...",4.309km (2.677 miles),71,305.909km (190.083 miles),"1’11.473 (Juan Pablo Montoya, 2004)","1’07.281 (Lewis Hamilton, 2018, qualifying three)",331kph (205.674 mph),195m,"1,394m"
7,"Circuit Gilles Villeneuve, Montreal – circuit ...",4.361km (2.71 miles),70,305.27km (189.686 miles),"1’13.622 (Rubens Barrichello, 2004)","1’10.764 (Sebastian Vettel, 2018, qualifying t...",315.5kph (196.043 mph),159.154m,1190m
8,Shanghai International Circuit information,5.451km,56,305.006km,"1’32.238 (Michael Schumacher, 2004)","1’31.095 (Sebastian Vettel, 2018, qualifying t...",348kph,336m,1397m
9,Silverstone – circuit information,5.891km (3.66 miles),52,306.332km (190.346 miles),"1’27.097 (Max Verstappen, 2020)","1’24.303 (Lewis Hamilton, 2020, qualifying three)",329.5kph (204.742 mph),296m,"1,034m"


In [ ]:
# This row was falsely extracted, most likely due to it having the same url pattern as the track information
# As we do not need this row we can remove it completely from the dataset
df.drop([33], inplace = True)

In [ ]:
df.head()

,Circuit Name,Lap length,Race laps,Race distance,Lap record*,Fastest lap,Maximum speed,Distance from grid to turn one,Longest flat-out section
0,"Yas Marina, Abu Dhabi – circuit information",5.554km (3.451 miles),55,305.355km (189.739 miles),"1’40.279 (Sebastian Vettel, 2009)","1’34.779 (Lewis Hamilton, 2019, qualifying three)",339kph (210.645 mph),305m,1233m
1,"Albert Park, Melbourne – circuit information",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Red Bull Ring – circuit information,4.326km (2.688 miles),71,307.02km (190.773 miles),"1’05.609 (Carlos Sainz Jnr, 2020)","1’02.939 (Valtteri Bottas, 2020, qualifying th...",327.4kph (203.437 mph),216.94m,868m
3,Baku City Circuit – track information,6.003km (3.73 miles),51,306.051km (190.171 miles),"1’43.009 (Charles Leclerc, 2019)","1’40.495 (Valtteri Bottas, 2019, qualifying th...",337kph (209.402 mph),137.8m,2010m
4,Bahrain International Circuit – track information,5.412km (3.363 miles),57,308.238km (191.53 miles),"1’31.447 (Pedro de la Rosa, 2005)","1’27.264 (Lewis Hamilton, 2020, qualifying three)",329.6kph (204.804 mph),336.49m,1205m


In [ ]:
# Dropping Albert Park as there is no information about this circuit on this website
df.drop([1], inplace = True)

In [ ]:
df

,Circuit Name,Lap length,Race laps,Race distance,Lap record*,Fastest lap,Maximum speed,Distance from grid to turn one,Longest flat-out section
0,"Yas Marina, Abu Dhabi – circuit information",5.554km (3.451 miles),55,305.355km (189.739 miles),"1’40.279 (Sebastian Vettel, 2009)","1’34.779 (Lewis Hamilton, 2019, qualifying three)",339kph (210.645 mph),305m,1233m
2,Red Bull Ring – circuit information,4.326km (2.688 miles),71,307.02km (190.773 miles),"1’05.609 (Carlos Sainz Jnr, 2020)","1’02.939 (Valtteri Bottas, 2020, qualifying th...",327.4kph (203.437 mph),216.94m,868m
3,Baku City Circuit – track information,6.003km (3.73 miles),51,306.051km (190.171 miles),"1’43.009 (Charles Leclerc, 2019)","1’40.495 (Valtteri Bottas, 2019, qualifying th...",337kph (209.402 mph),137.8m,2010m
4,Bahrain International Circuit – track information,5.412km (3.363 miles),57,308.238km (191.53 miles),"1’31.447 (Pedro de la Rosa, 2005)","1’27.264 (Lewis Hamilton, 2020, qualifying three)",329.6kph (204.804 mph),336.49m,1205m
5,Spa-Francorchamps – circuit information,7.004km (4.352 miles),44,308.052km (191.415 miles),"1’46.286 (Valtteri Bottas, 2018)","1’41.252 (Lewis Hamilton, 2020, qualifying three)",333kph (206.917mph),271m,2015m
6,"Interlagos, Autodromo Jose Carlos Pace circuit...",4.309km (2.677 miles),71,305.909km (190.083 miles),"1’11.473 (Juan Pablo Montoya, 2004)","1’07.281 (Lewis Hamilton, 2018, qualifying three)",331kph (205.674 mph),195m,"1,394m"
7,"Circuit Gilles Villeneuve, Montreal – circuit ...",4.361km (2.71 miles),70,305.27km (189.686 miles),"1’13.622 (Rubens Barrichello, 2004)","1’10.764 (Sebastian Vettel, 2018, qualifying t...",315.5kph (196.043 mph),159.154m,1190m
8,Shanghai International Circuit information,5.451km,56,305.006km,"1’32.238 (Michael Schumacher, 2004)","1’31.095 (Sebastian Vettel, 2018, qualifying t...",348kph,336m,1397m
9,Silverstone – circuit information,5.891km (3.66 miles),52,306.332km (190.346 miles),"1’27.097 (Max Verstappen, 2020)","1’24.303 (Lewis Hamilton, 2020, qualifying three)",329.5kph (204.742 mph),296m,"1,034m"
10,Hungaroring – circuit information,4.381km (2.722 miles),70,306.67km (190.556 miles),"1’17.103 (Max Verstappen, 2019)","1’13.447 (Lewis Hamilton, 2020, qualifying three)",315kph (195.732 mph),618m,908m


In [ ]:
# All the missing values are now due to missing data, not errors during webscraping
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33 entries, 0 to 34
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Circuit Name                    33 non-null     object
 1   Lap length                      33 non-null     object
 2   Race laps                       33 non-null     object
 3   Race distance                   33 non-null     object
 4   Lap record*                     27 non-null     object
 5   Fastest lap                     27 non-null     object
 6   Maximum speed                   30 non-null     object
 7   Distance from grid to turn one  26 non-null     object
 8   Longest flat-out section        22 non-null     object
dtypes: object(9)
memory usage: 2.6+ KB


In [ ]:
# Extract numeric part for km
df["Lap Length(km)"] = df["Lap length"].str.extract(r'(\d+\.\d+)').astype(float)

In [ ]:
df

,Circuit Name,Lap length,Race laps,Race distance,Lap record*,Fastest lap,Maximum speed,Distance from grid to turn one,Longest flat-out section,Lap Length(km)
0,"Yas Marina, Abu Dhabi – circuit information",5.554km (3.451 miles),55,305.355km (189.739 miles),"1’40.279 (Sebastian Vettel, 2009)","1’34.779 (Lewis Hamilton, 2019, qualifying three)",210.645,305m,1233m,5.554
2,Red Bull Ring – circuit information,4.326km (2.688 miles),71,307.02km (190.773 miles),"1’05.609 (Carlos Sainz Jnr, 2020)","1’02.939 (Valtteri Bottas, 2020, qualifying th...",327.400,216.94m,868m,4.326
3,Baku City Circuit – track information,6.003km (3.73 miles),51,306.051km (190.171 miles),"1’43.009 (Charles Leclerc, 2019)","1’40.495 (Valtteri Bottas, 2019, qualifying th...",209.402,137.8m,2010m,6.003
4,Bahrain International Circuit – track information,5.412km (3.363 miles),57,308.238km (191.53 miles),"1’31.447 (Pedro de la Rosa, 2005)","1’27.264 (Lewis Hamilton, 2020, qualifying three)",329.600,336.49m,1205m,5.412
5,Spa-Francorchamps – circuit information,7.004km (4.352 miles),44,308.052km (191.415 miles),"1’46.286 (Valtteri Bottas, 2018)","1’41.252 (Lewis Hamilton, 2020, qualifying three)",206.917,271m,2015m,7.004
6,"Interlagos, Autodromo Jose Carlos Pace circuit...",4.309km (2.677 miles),71,305.909km (190.083 miles),"1’11.473 (Juan Pablo Montoya, 2004)","1’07.281 (Lewis Hamilton, 2018, qualifying three)",205.674,195m,"1,394m",4.309
7,"Circuit Gilles Villeneuve, Montreal – circuit ...",4.361km (2.71 miles),70,305.27km (189.686 miles),"1’13.622 (Rubens Barrichello, 2004)","1’10.764 (Sebastian Vettel, 2018, qualifying t...",315.500,159.154m,1190m,4.361
8,Shanghai International Circuit information,5.451km,56,305.006km,"1’32.238 (Michael Schumacher, 2004)","1’31.095 (Sebastian Vettel, 2018, qualifying t...",NaN,336m,1397m,5.451
9,Silverstone – circuit information,5.891km (3.66 miles),52,306.332km (190.346 miles),"1’27.097 (Max Verstappen, 2020)","1’24.303 (Lewis Hamilton, 2020, qualifying three)",329.500,296m,"1,034m",5.891
10,Hungaroring – circuit information,4.381km (2.722 miles),70,306.67km (190.556 miles),"1’17.103 (Max Verstappen, 2019)","1’13.447 (Lewis Hamilton, 2020, qualifying three)",195.732,618m,908m,4.381


In [ ]:
df.dtypes

,0
Circuit Name,object
Lap length,object
Race laps,int64
Race distance,object
Lap record*,object
Fastest lap,object
Maximum speed,float64
Distance from grid to turn one,object
Longest flat-out section,object
Lap Length(km),float64


In [ ]:
# Converting race laps to integers
df['Race laps']=df['Race laps'].astype(int)

In [ ]:
df

,Circuit Name,Lap length,Race laps,Race distance,Lap record*,Fastest lap,Maximum speed,Distance from grid to turn one,Longest flat-out section,Lap Length(km),Lap times
0,"Yas Marina, Abu Dhabi – circuit information",5.554km (3.451 miles),55,305.355km (189.739 miles),"1’40.279 (Sebastian Vettel, 2009)","1’34.779 (Lewis Hamilton, 2019, qualifying three)",210.645,305m,1233m,5.554,"1’40.279 (Sebastian Vettel, 2009)"
2,Red Bull Ring – circuit information,4.326km (2.688 miles),71,307.02km (190.773 miles),"1’05.609 (Carlos Sainz Jnr, 2020)","1’02.939 (Valtteri Bottas, 2020, qualifying th...",327.400,216.94m,868m,4.326,"1’05.609 (Carlos Sainz Jnr, 2020)"
3,Baku City Circuit – track information,6.003km (3.73 miles),51,306.051km (190.171 miles),"1’43.009 (Charles Leclerc, 2019)","1’40.495 (Valtteri Bottas, 2019, qualifying th...",209.402,137.8m,2010m,6.003,"1’43.009 (Charles Leclerc, 2019)"
4,Bahrain International Circuit – track information,5.412km (3.363 miles),57,308.238km (191.53 miles),"1’31.447 (Pedro de la Rosa, 2005)","1’27.264 (Lewis Hamilton, 2020, qualifying three)",329.600,336.49m,1205m,5.412,"1’31.447 (Pedro de la Rosa, 2005)"
5,Spa-Francorchamps – circuit information,7.004km (4.352 miles),44,308.052km (191.415 miles),"1’46.286 (Valtteri Bottas, 2018)","1’41.252 (Lewis Hamilton, 2020, qualifying three)",206.917,271m,2015m,7.004,"1’46.286 (Valtteri Bottas, 2018)"
6,"Interlagos, Autodromo Jose Carlos Pace circuit...",4.309km (2.677 miles),71,305.909km (190.083 miles),"1’11.473 (Juan Pablo Montoya, 2004)","1’07.281 (Lewis Hamilton, 2018, qualifying three)",205.674,195m,"1,394m",4.309,"1’11.473 (Juan Pablo Montoya, 2004)"
7,"Circuit Gilles Villeneuve, Montreal – circuit ...",4.361km (2.71 miles),70,305.27km (189.686 miles),"1’13.622 (Rubens Barrichello, 2004)","1’10.764 (Sebastian Vettel, 2018, qualifying t...",315.500,159.154m,1190m,4.361,"1’13.622 (Rubens Barrichello, 2004)"
8,Shanghai International Circuit information,5.451km,56,305.006km,"1’32.238 (Michael Schumacher, 2004)","1’31.095 (Sebastian Vettel, 2018, qualifying t...",NaN,336m,1397m,5.451,"1’32.238 (Michael Schumacher, 2004)"
9,Silverstone – circuit information,5.891km (3.66 miles),52,306.332km (190.346 miles),"1’27.097 (Max Verstappen, 2020)","1’24.303 (Lewis Hamilton, 2020, qualifying three)",329.500,296m,"1,034m",5.891,"1’27.097 (Max Verstappen, 2020)"
10,Hungaroring – circuit information,4.381km (2.722 miles),70,306.67km (190.556 miles),"1’17.103 (Max Verstappen, 2019)","1’13.447 (Lewis Hamilton, 2020, qualifying three)",195.732,618m,908m,4.381,"1’17.103 (Max Verstappen, 2019)"


In [ ]:
df.to_csv('additional information circuits.csv', index=False)